In [1]:
import gym, time, random, numpy as np
from IPython.display import clear_output
from gym.envs.registration import register
try:
    register(
        id='FrozenLakeNoSlip-v0',
        entry_point='gym.envs.toy_text:FrozenLakeEnv',
        kwargs={'map_name' : '4x4', 'is_slippery' : False},
        max_episode_steps=100,
        reward_threshold=0.7, # optimum = 0.74 Note: look into this tomorrow.
    )
except:
    pass
envName = "FrozenLakeNoSlip-v0"
env = gym.make(envName)

In [15]:
class Agent():
    def __init__(self, env):
        self.actionSize = env.action_space.n
    
    def getAction(self):
        action = random.randint(0, self.actionSize - 1)
        return action
    


class qAgent(Agent):
    def __init__(self, env, discount, learningRate):
        super().__init__(env)
        self.stateSize = env.observation_space.n
        self.makeqTable()
        self.discount = discount
        self.learningRate = learningRate
        self.eps = 1.0

    def makeqTable(self):
        self.qTable = 1e-4*np.zeros([self.stateSize, self.actionSize])
    
    def train(self, experience):
        state, action, nextState, reward, done = experience

        qNext = self.qTable[nextState]
        qNext = np.zeros([self.actionSize]) if done else qNext
        qTarget = reward + self.discount * np.max(qNext)

        qUpdate = qTarget - self.qTable[state, action]
        self.qTable[state, action] += self.learningRate * qUpdate
        if done:
            self.eps *= .997
        

    def getAction(self, state):
        qState = self.qTable[state]
        action = super().getAction() 
        greedy = np.argmax(qState)
        return action if random.random() < self.eps else greedy
agent = qAgent(env, .97, .01)
totRewards = 0


In [21]:

for _ in range(300):
    state = env.reset()
    done = False
    while not done:
        action = agent.getAction(state)
        nextState, reward, done, info = env.step(action)
        agent.train((state, action, nextState, reward, done))
        state = nextState
        print("state:",state,"eps:", agent.eps, "reward", totRewards)
        totRewards += reward
        env.render()
        time.sleep(0.05)
        print(agent.qTable)
        clear_output(wait = True)

state: 15 eps: 0.007904966614247262 reward 1232.0
  (Right)
SFFF
FHFH
FFFH
HFFG
[[5.93907863e-02 8.46996042e-01 1.63475762e-03 6.84043269e-02]
 [4.58141047e-02 0.00000000e+00 3.93928849e-07 6.36587536e-05]
 [1.60664228e-04 0.00000000e+00 0.00000000e+00 6.12893166e-11]
 [2.28910970e-13 0.00000000e+00 0.00000000e+00 0.00000000e+00]
 [7.17773979e-02 8.80701108e-01 0.00000000e+00 4.23422024e-02]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 1.07644184e-06]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00]
 [9.02502896e-02 0.00000000e+00 9.11218751e-01 6.98405557e-02]
 [7.23383302e-02 9.40553125e-01 6.53841138e-02 0.00000000e+00]
 [1.04070982e-02 3.84225764e-01 0.00000000e+00 6.96636351e-09]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 1.70425407e-01 9.69945478e-01 8.50668259e-02]
 [1.49700907e-01 2.13710013e-01 9.9999